# Example of ai agent approach using langchain

In [1]:
%pip install langchain-community langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [14]:
from dotenv import load_dotenv
load_dotenv()

True

### Example 1: llm with tools (no agent)

In [16]:
import os
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
)

@tool
def sum_all_numbers(numbers):
    """
    This tool takes a list of numbers and returns their sum.

    Args:
        numbers (list): A list of numbers to be summed.

    Returns:
        int: The sum of all numbers in the list.
    """
    return sum(numbers)

llm_with_tools = llm.bind_tools([sum_all_numbers])

user_input = "What is the sum of 1, 2, 3, 4, 5?"
result = llm_with_tools.invoke(user_input)
print(result.tool_calls)

[{'name': 'sum_all_numbers', 'args': {'numbers': [1, 2, 3, 4, 5]}, 'id': 'call_Srty144SijmeNNGiEgxopJD0', 'type': 'tool_call'}]


### Example 2: Tools Agent

In [25]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("human", "{input}"),
    ("human", "{agent_scratchpad}")
])

agent = create_tool_calling_agent(llm, [sum_all_numbers], prompt)
executor = AgentExecutor(agent=agent, tools=[sum_all_numbers])

# Change to streaming output
for chunk in executor.stream({"input": "What is the sum of 1, 2, 3, 4, 5?"}):
    print(chunk)


{'actions': [ToolAgentAction(tool='sum_all_numbers', tool_input={'numbers': [1, 2, 3, 4, 5]}, log="\nInvoking: `sum_all_numbers` with `{'numbers': [1, 2, 3, 4, 5]}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_XC9S0pbCWzOmCv3dyceZrkkE', 'function': {'arguments': '{"numbers":[1,2,3,4,5]}', 'name': 'sum_all_numbers'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_b705f0c291'}, id='run-18d291bf-60eb-4f59-854e-5f8e23504b3a', tool_calls=[{'name': 'sum_all_numbers', 'args': {'numbers': [1, 2, 3, 4, 5]}, 'id': 'call_XC9S0pbCWzOmCv3dyceZrkkE', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'sum_all_numbers', 'args': '{"numbers":[1,2,3,4,5]}', 'id': 'call_XC9S0pbCWzOmCv3dyceZrkkE', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_XC9S0pbCWzOmCv3dyceZrkkE')], 'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [